# Tweet similarity ranking

### Problem:

#1: Use Twitter API to collect 1000 tweets in which keyword ‘narendra modi’ appears, save the collected tweets in nm.txt 
#2: Convert the collected tweets into BoW vectors and find cosine similarity of a pair of tweets and print the top-10 most similar tweet pairs, print these pairs
#3: Do the same using TF-IDF vectors


In [1]:
import tweepy
import pandas as pd
import numpy as np

In [2]:
from secrets import *

In [3]:
# Connect twitter API

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth, wait_on_rate_limit=True)

In [4]:
data_path = './data/'

In [5]:
search_words = "narendra modi"
no_of_tweets = 1000

In [6]:
# Ignoring retweets to avoid duplicacy

new_search = "narendra modi -filter:retweets"

In [10]:
tweets = tweepy.Cursor(api.search,
                  q=new_search).items(no_of_tweets)

In [11]:
# Adding tweet text to nm.txt
def text_to_file(tweets, file_name):
    with open(data_path+file_name, "w") as f:
        for tweet in tweets:
            f.write(tweet.text + '\n\n ----------------------\n')

In [12]:
# Convert tweets data into dataframe

text_details = [[tweet.id, tweet.created_at, tweet.user.screen_name, tweet.text] for tweet in tweets]

tweet_text = pd.DataFrame(data=text_details,
                         columns=['Tweet_id', 'Created At', 'Created By', 'Tweet Text'])

tweet_text

,Tweet_id,Created At,Created By,Tweet Text
0,1320844827869630466,2020-10-26 21:48:41,Vishal70896723,@narendramodi Please narendra modi g tik tok o...
1,1320843065158844416,2020-10-26 21:41:41,Vishal70896723,Please narendra modi g tik tok or helo firse c...
2,1320842727685189632,2020-10-26 21:40:21,Vishal70896723,@narendramodi Please narendra modi g tik tok o...
3,1320842156643209216,2020-10-26 21:38:04,ShaafFarid1,"""is kahani ke sabhi patra kalpnik h aur uska v..."
4,1320842114327023617,2020-10-26 21:37:54,LAXMIKANTAMALI9,ଅନେକ ଅନେକ ଶୁଭେଚ୍ଛା ଓ ଅଭିନନ୍ଦନ ମନିଷା ଅପା ଆମ ସୋ...
5,1320842042226937859,2020-10-26 21:37:37,vijendrajaswal,Please do remember these hardships faced by Bi...
6,1320840569627828229,2020-10-26 21:31:46,Anitaloveslove,Diaspora Indians kowtowing white supremacy onl...
7,1320839227827908609,2020-10-26 21:26:26,rahulra99131936,#aadha adhura ews reservation#Bring ews par wi...
8,1320839093287223296,2020-10-26 21:25:54,RRcsk,@brajendakr @realjharahul @yadavtejashwi Where...
9,1320838995039776770,2020-10-26 21:25:31,ANJALI72348961,Narendra Modi had also readily agreed to come ...


In [13]:
tweet_text.shape

(1000, 4)